In [1]:
import pygame
from drone import Drone
from baseStationControl import BaseStationControl
from adhoc import AdHoc
from globals import *

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
fanet = AdHoc()

drone_0 = Drone()
drone_1 = Drone()
drone_2 = Drone()
drone_3 = Drone()
drone_4 = Drone()
drone_5 = Drone()
drone_6 = Drone()
drone_7 = Drone()
drone_8 = Drone()
drone_9 = Drone()
drone_10 = Drone()
drone_11 = Drone()
drone_12 = Drone()

base_station_0 = BaseStationControl(position=(-LARGURA * 0.9, -ALTURA * 0.9, 0))

fanet.add_drone([drone_0, drone_1, drone_2])
fanet.add_bsc([base_station_0])

In [3]:
base_station_0.send_msg((0, 0, 10), "discover")
# drone_0.goto((0, 0, 10))

AttributeError: 'BaseStationControl' object has no attribute 'target'

In [ ]:
_ = fanet.update()
print(f"fanet.messages_in_transit: {fanet.messages_in_transit}")

In [ ]:
# Configuração do pygame
pygame.init()
SCREEN = pygame.display.set_mode((LARGURA, ALTURA))
pygame.display.set_caption("Flying Ad Hoc Network (FANET)")

clock = pygame.time.Clock()
RUNNING = True
PAUSED = False

while RUNNING:
    delta_time = clock.tick(FPS) / 1000.0  # Tempo decorrido em segundos

    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            RUNNING = False
        if evento.type == pygame.KEYDOWN:
            if evento.key == pygame.K_SPACE:  # Alterna entre pausado e rodando
                PAUSED = not PAUSED

    if not PAUSED:
        SCREEN.fill(BLACK)
        PAUSED = fanet.update()

        for uav_id, uav in fanet.uav.items():
            uav.update()

        for bsc_id, bsc in fanet.bsc.items():
            bsc.update()

        PAUSED = fanet.draw(SCREEN) or PAUSED

    else:
        # Mensagem na tela enquanto pausado
        font = pygame.font.Font(None, 30)
        pause_text = font.render("Paused. Press Space to continue.", True, (255, 255, 255))
        SCREEN.blit(pause_text, (LARGURA // 2 - pause_text.get_width() // 2, ALTURA // 2 - pause_text.get_height() // 2))

    pygame.display.flip()

pygame.quit()
